In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import argparse
import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

2024-10-28 17:43:52.526620: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-28 17:43:52.539676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-28 17:43:52.559482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-28 17:43:52.565774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 17:43:52.580788: I tensorflow/core/platform/cpu_feature_guar

In [3]:
batch_size = 32
model_name_or_path = "roberta-large"
task = "sst2"
peft_type = PeftType.LORA
device = "cuda"
num_epochs = 20

In [4]:
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
lr = 3e-4

In [5]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

datasets = load_dataset("glue", task)
metric = evaluate.load("glue", task)


def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["sentence"], truncation=True, max_length=None)
    return outputs


tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=["idx", "sentence"],
)

# We also rename the 'label' column to 'labels' which is the expected name for labels by the models of the
# transformers library
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")


# Instantiate dataloaders.
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, collate_fn=collate_fn, batch_size=batch_size)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], shuffle=False, collate_fn=collate_fn, batch_size=batch_size
)

In [6]:
# datasets[]

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,838,082 || all params: 357,199,876 || trainable%: 0.5146


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [9]:
print_trainable_parameters(model)

trainable params: 1838082 || all params: 357199876 || trainable%: 0.51


In [10]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [11]:
from modules.clientSST import *

In [12]:
from torch.optim import AdamW
# Create clients (e.g., 3 clients)
num_clients = 3

# Get the total length of the dataset
total_len = len(tokenized_datasets["train"])

# Calculate the number of samples per client and the remainder
split_size = total_len // num_clients
remainder = total_len % num_clients

# Create list for splits
split_lengths = [split_size] * num_clients

# Distribute the remainder among the first few clients
for i in range(remainder):
    split_lengths[i] += 1

# Perform the split
# client_data_splits = torch.utils.data.random_split(train_dataloader, split_lengths)
train_dataloader = torch.utils.data.random_split(tokenized_datasets["train"], split_lengths)
client_data_splits = [DataLoader(train_dataloader[i], shuffle=True, collate_fn=collate_fn, batch_size=batch_size) for i in range(num_clients)]
clients = [ClientSST(client_id=i, model=copy.deepcopy(model), data=client_data_splits[i], device=device) for i in range(num_clients)]

In [13]:
from modules.server import *
server = Server(global_model=model, device=device)

In [14]:
import wandb
wandb.init(project="FeDORA Project", entity="quanla", name="LoRA")

10/28/2024 17:44:35:ERROR:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [15]:
# 6. Federated Training Loop
num_rounds = 30  # Number of federated learning rounds
epochs_per_client = 5  # Number of epochs per client in each round

for round_num in range(num_rounds):
    print(f"\n--- Federated Learning Round {round_num + 1} ---")
    
    client_models = []
    
    # Train each client locally
    for client in clients:
        print(f"\nTraining Client {client.client_id}")
        client.train(epochs=epochs_per_client)
        client_models.append(client.get_parameters())

    # Aggregate client models on the server
    print("\nAggregating client models on the server...")
    server.aggregate(client_models)
    eval_metric = server.evaluate(eval_dataloader, metric=metric)
    print(eval_metric)
    wandb.log(eval_metric)
    # evaluate(lora_model, val_dataset)

    
    # Update each client's model with the new global model
    for client in clients:
        client.set_parameters(server.global_model.state_dict())

wandb.finish()


--- Federated Learning Round 1 ---

Training Client 0


100%|██████████| 702/702 [00:41<00:00, 17.07it/s]



Training Client 1


100%|██████████| 702/702 [00:42<00:00, 16.39it/s]



Training Client 2


...

100%|██████████| 702/702 [00:42<00:00, 16.49it/s]



Aggregating client models on the server...


100%|██████████| 28/28 [00:00<00:00, 29.36it/s]


{'accuracy': 0.9575688073394495}


accuracy,▁▃▄▄▅▆▂▅▅▅▆▅▄▅▆▅▆▆▇▆█▆▅▆▆▆▆▅▅▅
accuracy,0.95757


In [16]:
# model.to(device)
# for epoch in range(num_epochs):
#     model.train()
#     for step, batch in enumerate(tqdm(train_dataloader)):
#         batch.to(device)
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()

#     model.eval()
#     for step, batch in enumerate(tqdm(eval_dataloader)):
#         batch.to(device)
#         with torch.no_grad():
#             outputs = model(**batch)
#         predictions = outputs.logits.argmax(dim=-1)
#         predictions, references = predictions, batch["labels"]
#         metric.add_batch(
#             predictions=predictions,
#             references=references,
#         )

#     eval_metric = metric.compute()
#     print(f"epoch {epoch}:", eval_metric)

In [17]:
torch.cuda.current_device()

0